In [0]:
# Working code to update/insert/Delete using python 
import dlt
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr
from datetime import datetime as dt
import json
from pyspark.sql.functions import get_json_object
from delta.tables import DeltaTable
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

spark.conf.set("spark.databricks.delta.properties.defaults.enableChangeDataFeed", True)

connectionString = "Endpoint=sb://eventhub-cloud.servicebus.windows.net/;SharedAccessKeyName=student-policy;SharedAccessKey=6r6Kn56jGQRjLgTZMBRi0YJckPiz9fbAj+AEhHzIcMY=;EntityPath=students"

ehConf = {}
ehConf["eventhubs.connectionString"] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
ehConf["eventhubs.consumerGroup"] = "$Default"

json_schema = StructType([
    StructField("table", StringType(), True),
    StructField("op_type", StringType(), True),
    StructField("op_ts", StringType(), True),
    StructField("current_ts", StringType(), True),
    StructField("pos", StringType(), True),
    StructField("after", StructType([
        StructField("STUDENT_ID", IntegerType(), True),
        StructField("FIRST_NAME", StringType(), True),
        StructField("LAST_NAME", StringType(), True),
        StructField("AGE", IntegerType(), True),  # Assuming AGE is an integer
        StructField("GENDER", StringType(), True),
        StructField("GRADE", StringType(), True)
    ]), True),
    StructField("before", StructType([
        StructField("STUDENT_ID", IntegerType(), True),
        StructField("FIRST_NAME", StringType(), True),
        StructField("LAST_NAME", StringType(), True),
        StructField("AGE", IntegerType(), True),  # Assuming AGE is an integer
        StructField("GENDER", StringType(), True),
        StructField("GRADE", StringType(), True)
    ]), True)
])

streaming_df = (
    spark.readStream.format("eventhubs")
    .options(**ehConf)
    .load()
)

@dlt.create_table(comment="Bronze")
def bronze():
    return (
        streaming_df.withColumn("body", F.from_json(F.col("body").cast("string"), json_schema))
        .select(
            F.col("body.after.STUDENT_ID").alias("STUDENT_ID"),
            F.col("body.after.FIRST_NAME").alias("FIRST_NAME"),
            F.col("body.after.LAST_NAME").alias("LAST_NAME"),
            F.col("body.after.AGE").alias("AGE"),
            F.col("body.after.GENDER").alias("GENDER"),
            F.col("body.after.GRADE").alias("GRADE")
        )
    )



Name,Type
STUDENT_ID,int
FIRST_NAME,string
LAST_NAME,string
AGE,int
GENDER,string
GRADE,string
